In [1]:
from google.cloud import bigquery
from google.oauth2 import service_account

In [10]:
# set credentials with oauth2
credentials = service_account.Credentials.from_service_account_file('/home/douglas/Downloads/weather-5deec6be7e9f.json')

#Alternatively,set service private key file as default

1. Set the GOOGLE_APPLICATION_CREDENTIALS environment variable to the full path to your service account private key file

 $ export GOOGLE_APPLICATION_CREDENTIALS=/path/to/key.json

 Then, use application default credentials. default() checks for the GOOGLE_APPLICATION_CREDENTIALS environment variable before all other checks, so this will always use the credentials you explicitly specify.


 source: https://google-auth.readthedocs.io/en/latest/user-guide.html

In [8]:
# what's the point of scope? probably not needed in this use case
scoped_credentials = credentials.with_scopes(['https://www/googleapis.com/auth/cloud-platform'])

In [9]:
# create client for weather project using above service acount credentials
client = bigquery.Client(project='weather-214817', credentials=credentials)

In [11]:
import pickle 
import pandas as pd

In [13]:
with open('/home/douglas/ds_projects/Predicting_Wildfire_Size/stations2001.pkl', 'rb') as picklefile:
    stations2001 = pickle.load(picklefile)

In [14]:
stations2001

array(['911977', '911907', '912850', ..., '727517', '690524', '722067'],
      dtype=object)

In [24]:
query = '''SELECT stn, mo, da, temp, wdsp, max as max_temp, prcp, thunder  
        FROM `bigquery-public-data.noaa_gsod.gsod2001` 
        WHERE stn in ('911977', '911907', '722067') order by stn, mo'''

In [25]:
query_job = client.query(query)

In [32]:
# returns a row iterator 
rows = query_job.result()

In [27]:
type(rows)

google.cloud.bigquery.table.RowIterator

In [33]:
test_df = rows.to_dataframe()

In [35]:
test_df.groupby('stn').size()

stn
722067     34
911907     31
911977    245
dtype: int64

In [37]:
test_df[test_df['stn'] == '722067'].sort_values(['mo', 'da'])
#nulls are 999.9 for wdsp, 99.99 for prcp

,stn,mo,da,temp,wdsp,max_temp,prcp,thunder
5,722067,11,14,59.7,9.1,64.4,99.99,0
4,722067,11,15,68.0,13.4,73.4,0.00,0
0,722067,11,16,69.3,11.1,75.2,0.00,0
11,722067,11,19,66.7,999.9,75.2,0.00,0
7,722067,11,20,70.4,7.4,77.0,0.00,0
6,722067,11,21,63.6,999.9,71.6,0.00,0
1,722067,11,22,67.6,999.9,73.4,0.00,0
8,722067,11,23,72.2,5.8,78.8,0.00,0
9,722067,11,26,74.2,999.9,80.6,0.00,0
2,722067,11,27,72.3,4.8,78.8,0.00,0


In [38]:
query1993 = '''SELECT stn, mo, da, temp, wdsp, max as max_temp, prcp, thunder  
        FROM `bigquery-public-data.noaa_gsod.gsod1993` '''
query_job = client.query(query1993)
rows = query_job.result()
weather1993 = rows.to_dataframe()

In [39]:
weather1993.shape

(2564874, 8)

In [40]:
weather1993['stn'].nunique()

9000

In [41]:
us_only_stations = pd.read_csv('/home/douglas/ds_projects/Predicting_Wildfire_Size/us_only_stations.csv', header=None, usecols=[0], names=['USAF'])

In [46]:
us_stations = us_only_stations['USAF'].unique()

In [50]:
type(us_stations[0])

str

In [53]:
# one big string - wouldn't be read as a list
us_stations1 = ','.join(us_stations)

In [56]:
# tuple gives the commas necessary
us_stations2 = tuple(us_stations)

In [58]:
query1999 = '''SELECT stn, year, mo, da, temp, stp, wdsp, max as max_temp, prcp, thunder  
        FROM `bigquery-public-data.noaa_gsod.gsod1999` 
        WHERE stn IN {} order by stn, mo'''.format(tuple(us_stations))

In [60]:
%%time
query1999 = '''SELECT stn, year, mo, da, temp, stp, wdsp, max as max_temp, prcp, thunder  
        FROM `bigquery-public-data.noaa_gsod.gsod1999` 
        WHERE stn IN {} order by stn, mo'''.format(tuple(us_stations))
query_job = client.query(query1999)
rows = query_job.result()
weather1999 = rows.to_dataframe()

CPU times: user 12.9 s, sys: 313 ms, total: 13.2 s
Wall time: 53.4 s


4474

In [66]:
# 
len(weather1999['stn'].unique()) / len(us_stations)

0.33638801966919984

In [ ]:
%%time
query1999 = '''SELECT stn, year, mo, da, temp, stp, wdsp, max as max_temp, prcp, thunder  
        FROM `bigquery-public-data.noaa_gsod.gsod{}` 
        WHERE stn IN {} order by stn, mo'''.format('2000', tuple(us_stations))
query_job = client.query(query1999)
rows = query_job.result()
weather1999 = rows.to_dataframe()

In [68]:
# test 
query2000 = '''SELECT stn, year, mo, da, temp, stp, wdsp, max as max_temp, prcp, thunder  
        FROM `bigquery-public-data.noaa_gsod.gsod{}` 
        WHERE stn IN {} order by stn, mo'''.format('2000', tuple(us_stations))
#query2000

In [70]:
import numpy as np
np.arange(1992, 2016, 1)

array([1992, 1993, 1994, 1995, 1996, 1997, 1998, 1999, 2000, 2001, 2002,
       2003, 2004, 2005, 2006, 2007, 2008, 2009, 2010, 2011, 2012, 2013,
       2014, 2015])

In [72]:
df_names = ['weather' + str(yr) for yr in np.arange(1992,2016,1)]


In [73]:
queries = ['''SELECT stn, year, mo, da, temp, stp, wdsp, max as max_temp, prcp, thunder  
        FROM `bigquery-public-data.noaa_gsod.gsod{}` 
        WHERE stn IN {} order by stn, mo'''.format(str(yr), tuple(us_stations)) for yr in np.arange(1992,2016)]

In [75]:
len(queries)

24

In [77]:
%%time 
weather_dfs = dict()
for year, query in zip(df_names, queries):
    query_job = client.query(query)
    rows = query_job.result()
    weather_dfs[year] = rows.to_dataframe()
    print(year, 'downloaded')
    

weather1992 downloaded
weather1993 downloaded
weather1994 downloaded
weather1995 downloaded
weather1996 downloaded
weather1997 downloaded
weather1998 downloaded
weather1999 downloaded
weather2000 downloaded
weather2001 downloaded
weather2002 downloaded
weather2003 downloaded
weather2004 downloaded
weather2005 downloaded
weather2006 downloaded
weather2007 downloaded
weather2008 downloaded
weather2009 downloaded
weather2010 downloaded
weather2011 downloaded
weather2012 downloaded
weather2013 downloaded
weather2014 downloaded
weather2015 downloaded
CPU times: user 7min 19s, sys: 11.3 s, total: 7min 30s
Wall time: 31min 59s


In [78]:
len(weather_dfs)

24

In [79]:
weather_dfs.keys()

dict_keys(['weather1992', 'weather1993', 'weather1994', 'weather1995', 'weather1996', 'weather1997', 'weather1998', 'weather1999', 'weather2000', 'weather2001', 'weather2002', 'weather2003', 'weather2004', 'weather2005', 'weather2006', 'weather2007', 'weather2008', 'weather2009', 'weather2010', 'weather2011', 'weather2012', 'weather2013', 'weather2014', 'weather2015'])

In [80]:
len(weather_dfs['weather2010'])

942930

In [82]:
%%time 
for name in df_names:
    df = weather_dfs[name]
    with open('/home/douglas/ds_projects/Predicting_Wildfire_Size/data/{}.pkl'.format(name), 'wb') as picklefile:
        pickle.dump(df, picklefile)
    print(name, 'saved')

weather1992 saved
weather1993 saved
weather1994 saved
weather1995 saved
weather1996 saved
weather1997 saved
weather1998 saved
weather1999 saved
weather2000 saved
weather2001 saved
weather2002 saved
weather2003 saved
weather2004 saved
weather2005 saved
weather2006 saved
weather2007 saved
weather2008 saved
weather2009 saved
weather2010 saved
weather2011 saved
weather2012 saved
weather2013 saved
weather2014 saved
weather2015 saved
CPU times: user 36.1 s, sys: 21.5 s, total: 57.6 s
Wall time: 47min 28s
